**Note:** Run this file only once, to run it again, delete [**'res/tables.sql'**](../res)

In [1]:
import pandas as pd

### Load Raw Data 

In [2]:
data_url = 'https://raw.githubusercontent.com/tahmeed14/classification-models-for-alcoholism/master/alcoholism_classification/Student%20Alcohol%20Consumption%20Merged.csv'
# data_url = '../res/data/raw.csv'
raw_df = pd.read_csv(data_url)
raw_df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [3]:
raw_df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

### Select useful columns as features for the model

In [4]:
selections = ['school', 'sex', 'age', 'address', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'nursery', 'internet', 'higher', 'famsize', 'studytime', 'failures', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']

selected_df = raw_df[selections]
selected_df.head()

,school,sex,age,address,Pstatus,Medu,Fedu,Mjob,Fjob,reason,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,A,4,4,at_home,teacher,course,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,T,1,1,at_home,other,course,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,T,1,1,at_home,other,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,T,4,2,health,services,home,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,T,3,3,other,other,home,...,4,3,2,1,2,5,4,6,10,10


In [5]:
selected_df.nunique()

school         2
sex            2
age            8
address        2
Pstatus        2
Medu           5
Fedu           5
Mjob           5
Fjob           5
reason         4
guardian       3
traveltime     4
nursery        2
internet       2
higher         2
famsize        2
studytime      4
failures       4
romantic       2
famrel         5
freetime       5
goout          5
Dalc           5
Walc           5
health         5
absences      35
G1            18
G2            17
G3            19
dtype: int64

### Columns Renamed with more Descriptive Titles

In [6]:
replacements = {
    'Dalc': 'weekday drinker', 'Walc': 'weekend drinker', 'famrel': 'family relationship',
    'goout': 'social activities', 'romantic': 'significant other', 'freetime': 'free time',  
    'studytime': 'study time', 'famsize': 'household size', 'sex': 'gender', 'address': 'home address',
    'Pstatus': 'parents cohabitation', 'Fedu': 'dads education', 'Medu': 'moms education',
    'Mjob': 'moms job', 'Fjob': 'dads job', 'reason': 'choice reason', 'nursery': 'nursery education',
    'higher': 'higher education', 'traveltime': 'travel time', 'internet': 'internet access',
    'G1': 'first periods grade', 'G2': 'second periods grade', 'G3': 'third periods grade'
} 

selected_df = selected_df.rename(columns=replacements)
selected_df.head()

,school,gender,age,home address,parents cohabitation,moms education,dads education,moms job,dads job,choice reason,...,family relationship,free time,social activities,weekday drinker,weekend drinker,health,absences,first periods grade,second periods grade,third periods grade
0,GP,F,18,U,A,4,4,at_home,teacher,course,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,T,1,1,at_home,other,course,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,T,1,1,at_home,other,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,T,4,2,health,services,home,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,T,3,3,other,other,home,...,4,3,2,1,2,5,4,6,10,10


### Drinker Column Addition

In [7]:
drinker = selected_df['weekday drinker'] + selected_df['weekend drinker']

# encode drinker values 
encoded_drinker = pd.Series(map(lambda x: 0 if x < 4 else 1, drinker), name='drinker')

selected_df = selected_df.join(encoded_drinker)
selected_df.head()

,school,gender,age,home address,parents cohabitation,moms education,dads education,moms job,dads job,choice reason,...,free time,social activities,weekday drinker,weekend drinker,health,absences,first periods grade,second periods grade,third periods grade,drinker
0,GP,F,18,U,A,4,4,at_home,teacher,course,...,3,4,1,1,3,6,5,6,6,0
1,GP,F,17,U,T,1,1,at_home,other,course,...,3,3,1,1,3,4,5,5,6,0
2,GP,F,15,U,T,1,1,at_home,other,other,...,3,2,2,3,3,10,7,8,10,1
3,GP,F,15,U,T,4,2,health,services,home,...,2,2,1,1,5,2,15,14,15,0
4,GP,F,16,U,T,3,3,other,other,home,...,3,2,1,2,5,4,6,10,10,0


In [8]:
set(selected_df['internet access'])

{'no', 'yes'}

### Index Column Addition

In [9]:
index_col = pd.Series(range(1, len(selected_df)+1), name='id')
selected_df = selected_df.join(index_col)

selected_df.head()

,school,gender,age,home address,parents cohabitation,moms education,dads education,moms job,dads job,choice reason,...,social activities,weekday drinker,weekend drinker,health,absences,first periods grade,second periods grade,third periods grade,drinker,id
0,GP,F,18,U,A,4,4,at_home,teacher,course,...,4,1,1,3,6,5,6,6,0,1
1,GP,F,17,U,T,1,1,at_home,other,course,...,3,1,1,3,4,5,5,6,0,2
2,GP,F,15,U,T,1,1,at_home,other,other,...,2,2,3,3,10,7,8,10,1,3
3,GP,F,15,U,T,4,2,health,services,home,...,2,1,1,5,2,15,14,15,0,4
4,GP,F,16,U,T,3,3,other,other,home,...,2,1,2,5,4,6,10,10,0,5


### General
    no = 0
    yes = 1
    
### school
    GP = 0
    MS = 1
   
### gender
    M = 0
    F = 1
    
### houshold size
    LE3 = 0
    GT3 = 1
    
### jobs
    at_home = 0
    health = 1
    teacher = 2
    services = 3
    other = 4
    
### home address
    R = 0
    U = 1
 
### choice reason
    course = 0
    home = 1
    other = 2
    reputation = 3 
    
### guardian
    mother = 0
    fathet = 1
    other = 2
    
### parents cohabitation
    A = 0
    T = 1

In [10]:
# replace these values: ['no', 'yes', 'GP', 'MS', 'LE3', 'GT3', 'M', 'F']
#               with:   [  0,    1,     0,    1,     0,     1,   0,   1]

encoded_df = selected_df.replace(['no', 'yes', 'GP', 'MS', 'LE3', 'GT3', 'M', 'F', 'R', 'U', 'A', 'T'], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
encoded_df = encoded_df.replace(['at_home', 'health', 'teacher', 'services', 'other'], [0, 1, 2, 3, 4])
encoded_df = encoded_df.replace(['course', 'home', 'other', 'reputation'], [0, 1, 2, 3])
encoded_df = encoded_df.replace(['mother', 'father', 'other'], [0, 1, 2])
encoded_df[:5]

,school,gender,age,home address,parents cohabitation,moms education,dads education,moms job,dads job,choice reason,...,social activities,weekday drinker,weekend drinker,health,absences,first periods grade,second periods grade,third periods grade,drinker,id
0,0,1,18,1,0,4,4,0,2,0,...,4,1,1,3,6,5,6,6,0,1
1,0,1,17,1,1,1,1,0,4,0,...,3,1,1,3,4,5,5,6,0,2
2,0,1,15,1,1,1,1,0,4,4,...,2,2,3,3,10,7,8,10,1,3
3,0,1,15,1,1,4,2,1,3,1,...,2,1,1,5,2,15,14,15,0,4
4,0,1,16,1,1,3,3,4,4,1,...,2,1,2,5,4,6,10,10,0,5


In [11]:
encoded_df.loc[32]

school                   0
gender                   0
age                     15
home address             0
parents cohabitation     1
moms education           4
dads education           3
moms job                 2
dads job                 0
choice reason            0
guardian                 0
travel time              1
nursery education        1
internet access          1
higher education         1
household size           1
study time               2
failures                 0
significant other        1
family relationship      4
free time                5
social activities        2
weekday drinker          1
weekend drinker          1
health                   5
absences                 0
first periods grade     17
second periods grade    16
third periods grade     16
drinker                  0
id                      33
Name: 32, dtype: int64

### Re-order the columns in the dataframes and save them as csv files

In [12]:
# Re-order the dataframes
columns = ['id', 'school', 'gender', 'age', 'home address', 'parents cohabitation', 'moms education', 'dads education', 'moms job', 'dads job', 'choice reason', 'guardian', 'travel time', 'nursery education', 'internet access', 'higher education', 'significant other', 'household size', 'family relationship', 'free time', 'social activities', 'study time', 'first periods grade', 'second periods grade', 'third periods grade','failures', 'absences', 'health', 'weekday drinker', 'weekend drinker', 'drinker']
selected_df = selected_df[columns]
encoded_df = encoded_df[columns]

In [13]:
selected_df.head()

,id,school,gender,age,home address,parents cohabitation,moms education,dads education,moms job,dads job,...,study time,first periods grade,second periods grade,third periods grade,failures,absences,health,weekday drinker,weekend drinker,drinker
0,1,GP,F,18,U,A,4,4,at_home,teacher,...,2,5,6,6,0,6,3,1,1,0
1,2,GP,F,17,U,T,1,1,at_home,other,...,2,5,5,6,0,4,3,1,1,0
2,3,GP,F,15,U,T,1,1,at_home,other,...,2,7,8,10,3,10,3,2,3,1
3,4,GP,F,15,U,T,4,2,health,services,...,3,15,14,15,0,2,5,1,1,0
4,5,GP,F,16,U,T,3,3,other,other,...,2,6,10,10,0,4,5,1,2,0


In [14]:
encoded_df.head()

,id,school,gender,age,home address,parents cohabitation,moms education,dads education,moms job,dads job,...,study time,first periods grade,second periods grade,third periods grade,failures,absences,health,weekday drinker,weekend drinker,drinker
0,1,0,1,18,1,0,4,4,0,2,...,2,5,6,6,0,6,3,1,1,0
1,2,0,1,17,1,1,1,1,0,4,...,2,5,5,6,0,4,3,1,1,0
2,3,0,1,15,1,1,1,1,0,4,...,2,7,8,10,3,10,3,2,3,1
3,4,0,1,15,1,1,4,2,1,3,...,3,15,14,15,0,2,5,1,1,0
4,5,0,1,16,1,1,3,3,4,4,...,2,6,10,10,0,4,5,1,2,0


### CSV storage of datasets

In [15]:
raw_df.to_csv('../res/data/raw.csv', index=False)
selected_df.to_csv('../res/data/selected.csv', index=False)
encoded_df.to_csv('../res/data/encoded.csv', index=False)

## Database

In [16]:
import sqlite3
import numpy as np

con = sqlite3.connect('../res/data/tables.sql')
sqlite3.register_adapter(np.int32, int)
sqlite3.register_adapter(np.int64, int)
cur = con.cursor()

### Create database tables

In [17]:
profile_columns = ['id', 'school', 'gender', 'age', 'home address', 'parents cohabitation', 'moms education', 'dads education', 'choice reason', 'guardian', 'nursery education', 'higher education', 'significant other','household size', 'family relationship', 'health']
activities_columns = ['id', 'moms job', 'dads job', 'study time', 'travel time', 'internet access', 'absences', 'first periods grade', 'second periods grade', 'third periods grade', 'free time', 'social activities', 'failures', 'weekday drinker', 'weekend drinker', 'drinker']

cur.execute('CREATE TABLE profile ( id INT NOT NULL PRIMARY KEY ASC, school NUMERIC, gender NUMERIC, age NUMERIC, "home address" NUMERIC, "parents cohabitation" NUMERIC, "moms education" NUMERIC, "dads education" NUMERIC, "choice reason" NUMERIC, guardian NUMERIC, "nursery education" NUMERIC, "higher education" NUMERIC, "significant other" NUMERIC, "household size" NUMERIC, "family relationship" NUMERIC, health NUMERIC)')
cur.execute('CREATE TABLE activities (id INT NOT NULL PRIMARY KEY ASC, "moms job" NUMERIC, "dads job" NUMERIC, "study time" INT, "travel time" NUMERIC, "internet access" NUMERIC, absences INT, "first periods grade" NUMERIC, "second periods grade" NUMERIC, "third periods grade" NUMERIC, "free time" INT, "social activities" INT, failures INT, "weekday drinker" INT, "weekend drinker" INT, drinker INT)')
cur.execute('CREATE TABLE profile_encoded ( id INT NOT NULL PRIMARY KEY ASC, school NUMERIC, gender NUMERIC, age NUMERIC, "home address" NUMERIC, "parents cohabitation" NUMERIC, "moms education" NUMERIC, "dads education" NUMERIC, "choice reason" NUMERIC, guardian NUMERIC, "nursery education" NUMERIC, "higher education" NUMERIC, "significant other" NUMERIC, "household size" NUMERIC, "family relationship" NUMERIC, health NUMERIC)')
cur.execute('CREATE TABLE activities_encoded (id INT NOT NULL PRIMARY KEY ASC, "moms job" NUMERIC, "dads job" NUMERIC, "study time" INT, "travel time" NUMERIC, "internet access" NUMERIC, absences INT, "first periods grade" NUMERIC, "second periods grade" NUMERIC, "third periods grade" NUMERIC, "free time" INT, "social activities" INT, failures INT, "weekday drinker" INT, "weekend drinker" INT, drinker INT)')
con.commit()

### Populate tables

In [19]:
profile_encoded_values = encoded_df[profile_columns]
activities_encoded_values = encoded_df[activities_columns]
profile_values = selected_df[profile_columns]
activities_values = selected_df[activities_columns]

cur.executemany('INSERT INTO profile VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', profile_values.to_numpy())
cur.executemany('INSERT INTO activities VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', activities_values.to_numpy())
cur.executemany('INSERT INTO profile_encoded VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', profile_encoded_values.to_numpy())
cur.executemany('INSERT INTO activities_encoded VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', activities_encoded_values.to_numpy())

con.commit()
con.close()

The sqlite file **tables.sql** containing the tables  in the [resources directory](../res/data/) can viewed at [sqlite viewer](https://inloop.github.io/sqlite-viewer/)